# Modifying the motion function <a class='tocSkip'></a>

Thomas Schanzer z5310829  
School of Physics, UNSW  
September 2021

In this notebook, we modify the function currently used to calculate parcel motion, adding the ability to detect when the parcel reaches its neutral buoyancy level and its minimum height or the ground. This will later allow us to, for example, calculate the kinetic energy of a downdraft as it reaches the surface.

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Does-the-parcel-reach-the-surface?-If-so,-when?" data-toc-modified-id="Does-the-parcel-reach-the-surface?-If-so,-when?-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Does the parcel reach the surface? If so, when?</a></span></li></ul></div>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import metpy.calc as mpcalc
import metpy.constants as const
from metpy.units import units
from metpy.units import concatenate
from metpy.plots import SkewT

from scipy.interpolate import interp1d
from scipy.integrate import solve_ivp

from os import mkdir
from os.path import exists
import sys

from functools import partial

from environment import Environment

## Does the parcel reach the surface? If so, when?

We are interested in the properties of downdrafts when they reach the surface. We want to know whether, and at what time, the following events occur:
- Buoyancy is zero: parcel has reached neutral buoyancy level
    - What are the height and velocity at this time?
- Velocity is zero: parcel has reached minimum height without reaching ground; stop integration
    - What is the height at this time?
- Height is zero: parcel has hit ground; stop integration
    - What is the velocity at this time?

In [2]:
file = '../soundings/SYDNEY AIRPORT (94767) 12 Nov 2019 00Z.txt'
sounding = pd.read_csv(
    file, names=['pressure', 'temperature', 'dewpoint'],
    usecols=[0, 2, 3], header=0)
sounding = sounding.to_numpy()
pressure = sounding[:,0]*units.mbar
temperature = sounding[:,1]*units.celsius
dewpoint = sounding[:,2]*units.celsius

sydney = Environment(pressure, temperature, dewpoint)

In [136]:
class MotionResult():
    """
    Class for results of parcel motion calculations.
    
    Attributes:
        height: An array of heights, with each row corresponding to a
            different initial condition.
        velocity: An array of velocitites, with each row corresponding
            to a different initial condition.
        neutral_buoyancy_time: Times at which parcels reached their
            neutral buoyancy levels.
        hit_ground_time: Times at which parcels hit the ground.
        min_height_time: Times at which parcels reached their minimum
            heights (without hitting the ground)
        neutral_buoyancy_height: The heights of the neutral buoyancy
            levels.
        neutral_buoyancy_velocity: The velocities at the neutral
            buoyancy levels.
        hit_ground_velocity: The velocities with which the parcels hit
            the ground.
        min_height: The minimum heights of the parcels (that did not hit
            the ground).
    """
    
    def __init__(
            self, height, velocity, neutral_buoyancy_time, hit_ground_time,
            min_height_time, neutral_buoyancy_height,
            neutral_buoyancy_velocity, hit_ground_velocity, min_height):
        """
        Instantiates a MotionResult.
        """
        
        self.height = height
        self.velocity = velocity
        self.neutral_buoyancy_time = neutral_buoyancy_time
        self.hit_ground_time = hit_ground_time
        self.min_height_time = min_height_time
        self.neutral_buoyancy_height = neutral_buoyancy_height
        self.neutral_buoyancy_velocity = neutral_buoyancy_velocity
        self.hit_ground_velocity = hit_ground_velocity
        self.min_height = min_height

In [140]:
def modified_motion(time, initial_height, dq):
    """
    Calculates parcel motion for different specific humidity changes.

    Args:
        time: Array of time points.
        dq_range: Array of initial changes in specific humidity due
            to evaporation.
        initial_height: Initial height of the parcels.

    Returns:
        A MotionResult object.
    """
    
    # independent variables
    initial_height = np.atleast_1d(initial_height).to(units.meter).m
    dq = np.atleast_1d(dq)
    if len(initial_height) != len(dq):
        raise ValueError(
            'Initial height and specific humidity change arrays must '
            'have the same length.')
    initial_state = [[z0, 0] for z0 in initial_height]
    time = time.to(units.second).m

    def motion_ode(
            time, state, initial_height, dq):
        """
        Defines the equation of motion for a parcel.
        """
        
        buoyancy = sydney.parcel_buoyancy(
            state[0]*units.meter, initial_height*units.meter, dq)
        return [state[1], buoyancy.magnitude]
    
    # event function for solve_ivp, zero when parcel reaches min height
    min_height = lambda time, state, *args: state[1]
    min_height.direction = 1  # find zero that goes from negative to positive
    min_height.terminal = True  # stop integration at minimum height
    
    # event function for solve_ivp, zero when parcel hits ground
    hit_ground = lambda time, state, *args: state[0]
    hit_ground.terminal = True  # stop integration at ground

    # prepare empty arrays for data
    height = np.zeros((len(dq), len(time)))
    height[:] = np.nan
    velocity = np.zeros((len(dq), len(time)))
    velocity[:] = np.nan
    
    neutral_buoyancy_time = np.zeros(len(dq))
    hit_ground_time = np.zeros(len(dq))
    min_height_time = np.zeros(len(dq))
    
    neutral_buoyancy_height = np.zeros(len(dq))
    neutral_buoyancy_velocity = np.zeros(len(dq))
    hit_ground_velocity = np.zeros(len(dq))
    min_height_height = np.zeros(len(dq))
    
    for i in range(len(dq)):
        sys.stdout.write(
            '\rCalculating profile {0} of {1}.'
            '   '.format(i+1, len(dq)))
        
        # event function for solve_ivp, zero when parcel is neutrally
        # buoyant
        neutral_buoyancy = lambda time, state, *args: motion_ode(
            time, state, initial_height[i], dq[i])[1]
        
        sol = solve_ivp(
            motion_ode,
            [np.min(time), np.max(time)],
            initial_state[i],
            t_eval=time,
            args=(initial_height[i], dq[i]),
            events=[neutral_buoyancy, hit_ground, min_height])
        
        height[i,:len(sol.y[0,:])] = sol.y[0,:]
        velocity[i,:len(sol.y[1,:])] = sol.y[1,:]
        
        # record times of events
        # sol.t_events[i].size == 0 means the event did not occur
        neutral_buoyancy_time[i] = (  # record only the first instance
            sol.t_events[0][0] if sol.t_events[0].size > 0 else np.nan)
        hit_ground_time[i] = (
            sol.t_events[1][0] if sol.t_events[1].size > 0 else np.nan)
        min_height_time[i] = (
            sol.t_events[2][0] if sol.t_events[2].size > 0 else np.nan)
        
        # record states at event times
        neutral_buoyancy_height[i] = (  # record only the first instance
            sol.y_events[0][0,0] if sol.y_events[0].size > 0 else np.nan)
        neutral_buoyancy_velocity[i] = (  # record only the first instance
            sol.y_events[0][0,1] if sol.y_events[0].size > 0 else np.nan)
        hit_ground_velocity[i] = (
            sol.y_events[1][0,1] if sol.y_events[1].size > 0 else np.nan)
        min_height_height[i] = (
            sol.y_events[2][0,0] if sol.y_events[2].size > 0 else np.nan)
        
    result = MotionResult(
        np.squeeze(height)*units.meter,
        np.squeeze(velocity)*units.meter/units.second,
        np.squeeze(neutral_buoyancy_time)*units.second,
        np.squeeze(hit_ground_time)*units.second,
        np.squeeze(min_height_time)*units.second,
        np.squeeze(neutral_buoyancy_height)*units.meter,
        np.squeeze(neutral_buoyancy_velocity)*units.meter/units.second,
        np.squeeze(hit_ground_velocity)*units.meter/units.second,
        np.squeeze(min_height_height)*units.meter)

    return result

In [139]:
time = np.arange(0, 500, 10)*units.second
dq = np.arange(0, 4.51e-3, 0.5e-3)
initial_height = np.ones(len(dq))*2000*units.meter

result = modified_motion(time, initial_height, dq)

Calculating profile 10 of 10.   

In [133]:
result.neutral_buoyancy_time

Magnitude,[0.0 140.85571820012038 153.38815008148413 nan nan nan nan nan nan nan]
Units,second


In [134]:
result.neutral_buoyancy_height

Magnitude,[2000.0 1682.973055995609 1271.2990309441689 nan nan nan nan nan nan nan]
Units,meter
